## FDW Crop Production Data Profiling - Guinea

In [1]:
import os, sys, glob, json
from itertools import product, compress, chain
from functools import reduce
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
from tools import save_hdf, save_npz, load_npz, PrintAdminUnits, PlotAdminShapes
from tools import FDW_PD_Sweeper, FDW_PD_AvalTable, FDW_PD_Compiling, FDW_PD_GrainTypeAgg, FDW_PD_ValidateFnidName
from tools import FDW_PD_CreateAdminLink, FDW_PD_RatioAdminLink, FDW_PD_ConnectAdminLink
from tools_graphic import PlotBarProduction, PlotLinePAY, PlotHeatCropSystem, PlotHeatSeasonData
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None

In [2]:
# CPCV2 grain code ------------------------------ #
grain_code = pd.read_hdf('./data/crop/grain_cpcv2_code.hdf')
product_category = grain_code[['product', 'product_category']].set_index('product').to_dict()['product_category']
# ----------------------------------------------- #

# Load FEWS NET administrative boundaries ------- #
epsg = 'epsg:32628' # Guinea
fn_shapes = sorted(glob.glob('./data/shapefile/fewsnet/GN_Admin?_????.shp'))
shape_all = []
for fn in fn_shapes:
    name = fn[-18:-4]
    exec('%s = gpd.read_file("%s").to_crs("%s")' % (name, fn, epsg))
    exec('%s["area"] = %s["geometry"].area/10**6' % (name, name))
    exec('shape_all.append(%s)' % (name))
shape_all = pd.concat(shape_all, axis=0).reset_index(drop=True)
PrintAdminUnits(shape_all)
# ----------------------------------------------- #

# FDW API host address -------------------------- #
host = 'https://fdw.fews.net'
auth = tuple(json.loads(open('token.json', "r").read()))
parameters = {
    'format': 'json',
    'country': 'Guinea',
    'product': 'R011',
    'survey_type': 'crop:best'
}
endpoint = '/api/cropproductionindicatorvalue/'
response = requests.get(host + endpoint, auth=auth, params=parameters, proxies={})
response.raise_for_status()
df = pd.DataFrame.from_records(response.json())
df_origin = df.copy()
# ----------------------------------------------- #

# Manual Pre-processing before Sweeping --------- #
# 1. Default setting 
# a) None-type population group
df.loc[df['population_group'].isna(), 'population_group'] = 'none'
df.loc[df['population_group'] == '', 'population_group'] = 'none'
# ----------------------------------------------- #

# FDW Production Data Inspection ---------------- #
df, df_raw = FDW_PD_Sweeper(df)
table_dict = FDW_PD_AvalTable(df, shape_all)
# ----------------------------------------------- #

# FEWS NET Shapefile comparison ----------------- #
shape_used = pd.concat([GN_Admin1_1990, GN_Admin2_1990], axis=0)
PlotAdminShapes(shape_used, label=True)
# ----------------------------------------------- #

- FEWS NET admin shapefiles ------------------- #
| year	 | Admin1   | # units   | Admin2   | # units   |
| 1990	 | GN1990A1 | 8	| GN1990A2	| 34	|
| 2014	 | GN2014A1 | 8	| GN2014A2	| 34	|
----------------------------------------------- #
- Remove missing records ---------------------- #
Orignial data points: 426
426/426 "Quantity Produced" points are retained.
Current data points: 426

- Minor changes are applied.. ----------------- #

- Basic information --------------------------- #
Data period: 2008 - 2015
3 grain types are found: Fonio, Maize (Corn), Rice (Paddy)
1 seasons are found: Main harvest (09-01)
1 crop production system are found: none
Data sources include:
[1] L'Agence Nationale des Statistiques Agricoles et Alimentaires, Guinea --- Dynamique de production, Guinea
Administrative-1 fnids: 7
Administrative-2 fnids: 33
0 reporting units are found: 

- Stop process is triggerd -------------------- #
indicator          Area Harvested  Area Planted  Quantity Produced  Yield
sea

ValueError: *** Process is stopped ***

- Guinea crop seasonal calendar

![FEWS NET](https://fews.net/sites/default/files/styles/large/public/seasonal-calendar-guinea.png?itok=_ljnFxNc)

- FDW data consists of `GN1990A1` and `GN1990A2`.

| Year | Admin-1 | # units  | Admin-2  | # units |
| :---: | :----:  | :----:   | :----:   | :---:  |
| 1990	 | **`GN1990A1`** | 8	| **`GN1990A2`**	| 34	|
| 2014	 | GN2014A1 | 8	| GN2014A2	| 34	|

- Comparison between admin boundaries.

![image](https://github.com/chc-ucsb/gscd/blob/main/figures/GN_admin_shapes.png?raw=true)

- **FDW data consists of "Quantity Produced" for 8 years**.
- **`GN1990A2`** is used to represent current admin-level 2 crop data.
- Guinea has a single crop seasons: Main harvest (09-01).
- Guinea has no crop production system.
- Guinea has no population group.

In [6]:
# Define the latest shapefile ------------------- #
latest_level = 2
shape_latest = GN_Admin2_1990.copy().to_crs('epsg:4326')
# ----------------------------------------------- #

# # Validation of FNIDs and Names ----------------- #
# df = FDW_PD_ValidateFnidName(df, shape_used, shape_latest)
# # ----------------------------------------------- #

# # FDW Production Data Compiling ----------------- #
# area, prod = FDW_PD_Compiling(df, shape_used)
# area_all, prod_all = area.copy(), prod.copy()
# mdx_pss = area.columns.droplevel([0,1]).unique()
# # ----------------------------------------------- #